# Querying chpaf MoPo

In [1]:
import rdflib
import pandas as pd

# Set pandas to display wide columns
pd.set_option('max_colwidth', 1000)

g = rdflib.Graph()
g.parse("../examples/mopo_mvp.ttl")

<Graph identifier=Nb8cc24880a36453cae02d7c073eda02a (<class 'rdflib.graph.Graph'>)>

## Chapter 1 - Proposal for Abandonment

In [2]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?text ?proposal WHERE {

    ?ProposalActivity prov:used 
        ?IdentifierEntity, 
        ?ProceduralRequestEntity, 
        ?ProposalEntity.

    ?IdentifierEntity a chpaf:ParliamentaryAffairIdentifierEntity;
        schema:identifier ?id.
    
    ?ProceduralRequestEntity a chpaf:ProceduralRequestEntity;
        schema:title ?title;
        schema:description ?text.
    
    ?ProposalEntity a chpaf:ProceduralRequestProposalEntity;
        schema:description ?proposal.

} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,id,title,text,proposal
0,21.4382,"Eine Armee, die ihre Auswirkungen auf die Biodiversität senkt","Der Bundesrat, insbesondere das Departement für Verteidigung, Bevölkerungsschutz und Sport (VBS), wird beauftragt...",Das VBS hat seinen Aktionsplan Biodiversität am 22. Juni 2023 veröffentlicht [...] Der Bundesrat erachtet das Anliegen der Motion als erfüllt und beantragt deren Abschreibung.


## Chapter 2 - Information on the Fulfilment Status

In [3]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?title ?text ?information WHERE {

    ?InformationActivity prov:used 
        ?IdentifierEntity, 
        ?ProceduralRequestEntity, 
        ?InformationEntity.

    ?IdentifierEntity a chpaf:ParliamentaryAffairIdentifierEntity;
        schema:identifier ?id.
    
    ?ProceduralRequestEntity a chpaf:ProceduralRequestEntity;
        schema:title ?title;
        schema:description ?text.
    
    ?InformationEntity a chpaf:ProceduralRequestInformationEntity;
        schema:description ?information.

} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,id,title,text,information
0,21.3877,Evaluation der Strukturreform BVG,"Der Bundesrat wird beauftragt, die vor zehn Jahren in Kraft getretene Strukturreform BVG [...]","Die Forschungsarbeiten sind im Gang. Sie werden als Grundlage für den Bericht des Bundesrates dienen, der für die zweite Hälfte 2024 geplant ist."
1,21.3968,Zielerreichung der Strukturreform BVG evaluieren,"Der Bundesrat wird beauftragt, die vor zehn Jahren in Kraft getretene Strukturreform BVG [...] Eine Minderheit der Kommission beantragt, das Postulat abzulehnen.","Die Forschungsarbeiten sind im Gang. Sie werden als Grundlage für den Bericht des Bundesrates dienen, der für die zweite Hälfte 2024 geplant ist."


## Connex Procedural Requests

--> Does not work because symmetry does not allow to discern between `?main` and `?dep`

In [4]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?main ?dep WHERE {

    ?activity prov:used/schema:identifier ?dep;
        prov:used/^prov:used/prov:used/schema:identifier ?main.

    Filter(?main != ?dep)
} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,main,dep
0,21.3968,21.3877
1,21.3877,21.3968


if class `chpaf:ConnexActivity` is added to the corresponding activity, the query is possible:

In [5]:
query_string = """

PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?main ?dep WHERE {

    ?activity a chpaf:ConnexActivity;
        prov:used/schema:identifier ?dep;
        prov:used/^prov:used/prov:used/schema:identifier ?main.

    Filter(?main != ?dep)
} 

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,main,dep
0,21.3877,21.3968
